In [10]:
from dotenv import load_dotenv
from typing import Annotated
import os

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [11]:
# from langchain.chat_models import init_chat_model
from langchain_openai import AzureChatOpenAI
# llm = init_chat_model("google_genai:gemini-2.0-flash")
# llm.invoke("who was the person to walk on the moon")

openai_llm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    api_key=AZURE_OPENAI_API_KEY or "",
    azure_endpoint=AZURE_OPENAI_ENDPOINT or "",
    api_version="2024-12-01-preview",
    stream_usage=True,
)
# openai_llm.invoke("who was the person to walk on the moon")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()




In [13]:
# message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='What is the latest price of MSFT stock?', additional_kwargs={}, response_metadata={}, id='bc03559c-9923-4ba8-9660-592bdacd24c6'),
 AIMessage(content='As of November 1, 2024, at 4:31 PM PST, the latest price of MSFT (Microsoft) stock is $429.38.\n\nThis is according to the live data available from Google Finance.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--4d275f80-d09d-40e2-8ea5-5efc66a1d291-0', usage_metadata={'input_tokens': 10, 'output_tokens': 54, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})]